In [1]:
import torch
from transformers import *
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import re
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


I0531 17:52:00.970374 35000 file_utils.py:41] PyTorch version 1.1.0 available.


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

I0531 17:52:07.272499 35000 tokenization_utils.py:504] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at C:\Users\Jai\.cache\torch\transformers\5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [3]:
model = BertModel.from_pretrained('bert-base-cased', output_hidden_states = True)

I0531 17:52:07.555935 35000 configuration_utils.py:283] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at C:\Users\Jai\.cache\torch\transformers\b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
I0531 17:52:07.560920 35000 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,


In [4]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1)
          )
        )
        (in

In [50]:
CLEANED_NAMES_FILE = "../Data/category_cleaned_names.csv"
# brands = ["he", "him", "her", "masculine", "she", "girl", "father", "mother", "lettuce"]#, "she", "He", "She"]
# test = "This is a test embedding"

In [7]:
def get_embedding(brand, is_brand=True):
    if is_brand:
        brand = brand.capitalize()
    input_ids = torch.tensor(tokenizer.encode(brand)).unsqueeze(0)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        hidden_states = model(input_ids)[0][0].numpy()
    sum_vec = np.mean(hidden_states[1:-1], axis=0)
#     print(sum_vec.shape)
#     print(brand)
#     print(tokenizer.tokenize(brand.capitalize()))

#         hidden_states = model(input_ids)[2]
#     token_embeddings = torch.stack(hidden_states, dim=0)
#     token_embeddings = torch.squeeze(token_embeddings, dim=1)
#     token_embeddings = token_embeddings.permute(1,0,2)
#     print(token_embeddings.size())
#     for token in token_embeddings[1:-1]:
#         sum_vec = torch.sum(token[-4:], dim=0)
#     print(type(sum_vec))
#     print(sum_vec.numpy().shape)
    return sum_vec.reshape(1, -1)

In [17]:
def get_embedding_corporation(brand, is_brand=True):
    if is_brand:
        brand = brand.capitalize()
    brand = f"{brand} is a corporation"
    input_ids = torch.tensor(tokenizer.encode(brand)).unsqueeze(0)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        hidden_states = model(input_ids)[0][0].numpy()
        
    print(tokenizer.tokenize(brand))
#     print(hidden_states[1:-4].shape)
    sum_vec = np.mean(hidden_states[1:-4], axis=0)
#     print(sum_vec.shape)
#     print(brand)
    return sum_vec.reshape(1, -1)

In [54]:
def get_embedding_corporation_category(brand, category, is_brand=True):
    if is_brand:
        brand = brand.capitalize()
    brand = f"{brand} is a {category} corporation"
    input_ids = torch.tensor(tokenizer.encode(brand)).unsqueeze(0)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        hidden_states = model(input_ids)[0][0].numpy()
        
    whole_name = tokenizer.tokenize(brand)
    is_index = whole_name.index('is') + 1 #to account for CLS token
    print(hidden_states[1:is_index].shape)
    sum_vec = np.mean(hidden_states[1:is_index], axis=0)
#     print(sum_vec.shape)
    print(brand)
    return sum_vec.reshape(1, -1)

In [44]:
x = get_embedding_corporation_category('corporation', "tea and cookies")

In [9]:
he_she_axis = normalize(get_embedding('she', False) - get_embedding('he', False))

In [59]:
def get_gender_projection(brand, is_brand):
    return cosine_similarity(he_she_axis, get_embedding(brand, is_brand))[0][0]

def get_gender_projection_corporation(brand, is_brand):
    return cosine_similarity(he_she_axis, get_embedding_corporation(brand, is_brand))[0][0]

def get_gender_projection_corporation_category(brand, category, is_brand):
    return cosine_similarity(he_she_axis, get_embedding_corporation_category(brand, category, is_brand))[0][0]

def category_get_gender_projection(brand, category, is_brand):
    return cosine_similarity(he_she_axis, get_embedding(brand, is_brand) - get_embedding(category, is_brand))[0][0]

def category_get_gender_projection_corporation(brand, category, is_brand):
    return cosine_similarity(he_she_axis, get_embedding_corporation(brand, is_brand) - get_embedding(category, is_brand))[0][0]

def category_get_gender_projection_corporation_category(brand, category, is_brand):
    return cosine_similarity(he_she_axis, get_embedding_corporation_category(brand, category, is_brand) - get_embedding(category, is_brand))[0][0]

In [11]:
df_baseline = pd.read_csv('../Data/new_baseline.csv')
df_baseline['bert_alignment'] = df_baseline['name'].apply(lambda x : get_gender_projection(x, False))
print(pearsonr(df_baseline['new_alignment'], df_baseline['bert_alignment']))
df_baseline.sort_values(by='bert_alignment')

(0.022776175859299412, 0.8066030896411008)


,Unnamed: 0,Unnamed: 0.1,name,category,alignment,new_alignment,bert_alignment
44,480,108,buddies,BASELINE,-0.001285,-0.149317,-0.193535
61,745,4,fiancee,BASELINE,0.021869,0.125364,-0.184812
22,127,94,scrimmage,BASELINE,-0.060081,-0.119422,-0.166602
117,1586,46,sassy,BASELINE,0.302578,0.344233,-0.127684
65,854,96,zeal,BASELINE,0.032355,-0.041446,-0.114675
10,61,117,guru,BASELINE,-0.076805,-0.111828,0.002005
24,132,43,lamb,BASELINE,-0.058381,0.128258,0.012614
98,1465,70,tanning,BASELINE,0.134884,0.182024,0.022857
108,1560,60,flirt,BASELINE,0.219665,0.108724,0.046701
17,99,93,hoped,BASELINE,-0.065549,-0.035308,0.051942


In [60]:
df_brands = pd.read_csv('../Data/alignment_scores.csv')

df_brands_category = pd.read_csv('../Data/brand_names_coded.csv')
df_brands_no_dup = df_brands_category.drop_duplicates(subset='BRAND')
df_brands_no_dup = df_brands_no_dup[df_brands_no_dup['BRAND'].isin(df_brands['brand'].tolist())][['BRAND', 'CATEGORY']]
df_brands_no_dup

,BRAND,CATEGORY
0,adidas,A141 Sportswear & Athleticwear
1,alias,A141 Sportswear & Athleticwear
2,alpinestars,A141 Sportswear & Athleticwear
4,asics,A141 Sportswear & Athleticwear
5,assos,A141 Sportswear & Athleticwear
6,athleta,A141 Sportswear & Athleticwear
8,billabong,A141 Sportswear & Athleticwear
9,boathouse,A141 Sportswear & Athleticwear
10,bogner,A141 Sportswear & Athleticwear
11,calia,A141 Sportswear & Athleticwear


In [61]:
def preprocess(category):
    category = category[5:]
    category = re.sub(",|'s|& |/", ' ', category)
    category = re.sub("  ", " ", category )
    return category.lower()

def get_brand_categories(brands, concise=False):
    if concise:
        cleaned_name_map = pd.read_csv(CLEANED_NAMES_FILE).set_index('Original').to_dict()['Concise']
    else:
        cleaned_name_map = pd.read_csv(CLEANED_NAMES_FILE).set_index('Original').to_dict()['Corrected']
    valid_df = df_brands_no_dup[df_brands_no_dup['BRAND'].isin(brands)]
    brand_category_mapping = valid_df.set_index('BRAND').to_dict()['CATEGORY']
    brand_category_cleaned = {brand: preprocess(cleaned_name_map[category]) for brand, category in brand_category_mapping.items()}
#     brand_category_mapping = {brand: preprocess(category) for brand, category in brand_category_cleaned.items()}
    return brand_category_cleaned, list(set(brand_category_cleaned.values()))




In [62]:
brand_category_mapping, categories = get_brand_categories(df_brands['brand'].tolist())

In [63]:
df_brands['bert_alignment'] = df_brands['brand'].apply(lambda x: get_gender_projection(x, True))
df_brands['corp_alignment'] = df_brands['brand'].apply(lambda x: get_gender_projection_corporation(x, True))
df_brands['corp_cat_alignment'] = df_brands['brand'].apply(lambda x: 
                                                           get_gender_projection_corporation_category(x, 
                                                                                                      brand_category_mapping[x].lower(),
                                                                                                      True))
df_brands['cat_bert_alignment'] = df_brands['brand'].apply(lambda x: category_get_gender_projection(x,brand_category_mapping[x].lower(), True))
df_brands['cat_corp_alignment'] = df_brands['brand'].apply(lambda x: category_get_gender_projection_corporation(x, brand_category_mapping[x].lower(), True))
df_brands['cat_corp_cat_alignment'] = df_brands['brand'].apply(lambda x: 
                                                           category_get_gender_projection_corporation_category(x, 
                                                                                                      brand_category_mapping[x].lower(),
                                                                                                      True))

['Ever', '##bank', 'is', 'a', 'corporation']
['Great', '##s', 'is', 'a', 'corporation']
['Brothers', 'is', 'a', 'corporation']
['Li', '##ft', '##master', 'is', 'a', 'corporation']
['Ha', '##rp', '##oon', 'is', 'a', 'corporation']
['Johnson', 'is', 'a', 'corporation']
['Bo', '##oya', '##h', 'is', 'a', 'corporation']
['Ram', '##page', 'is', 'a', 'corporation']
['Re', '##eb', '##ok', 'is', 'a', 'corporation']
['E', '##sp', 'is', 'a', 'corporation']
['Jay', '##co', 'is', 'a', 'corporation']
['Cobra', 'is', 'a', 'corporation']
['Dai', '##wa', 'is', 'a', 'corporation']
['Bo', '##sch', 'is', 'a', 'corporation']
['A', '##win', 'is', 'a', 'corporation']
['Lands', '##har', '##k', 'is', 'a', 'corporation']
['Nike', 'is', 'a', 'corporation']
['J', '##b', '##m', 'is', 'a', 'corporation']
['Must', '##ad', 'is', 'a', 'corporation']
['Marco', 'is', 'a', 'corporation']
['Shi', '##man', '##o', 'is', 'a', 'corporation']
['Luc', '##ches', '##e', 'is', 'a', 'corporation']
['Gill', 'is', 'a', 'corporation']

['M', '##c', '##m', 'is', 'a', 'corporation']
['Stuart', 'is', 'a', 'corporation']
['Da', '##p', 'is', 'a', 'corporation']
['Via', '##nney', 'is', 'a', 'corporation']
['Co', '##rte', '##ch', 'is', 'a', 'corporation']
['Norris', 'is', 'a', 'corporation']
['Ser', '##eng', '##eti', 'is', 'a', 'corporation']
['Ke', '##russ', '##o', 'is', 'a', 'corporation']
['Mont', '##rail', 'is', 'a', 'corporation']
['Adams', 'is', 'a', 'corporation']
['Am', '##ax', 'is', 'a', 'corporation']
['Ra', '##uck', '##man', 'is', 'a', 'corporation']
['T', '##ro', '##eg', '##s', 'is', 'a', 'corporation']
['B', '##lou', '##nt', 'is', 'a', 'corporation']
['Em', '##erse', 'is', 'a', 'corporation']
['Direct', '##v', 'is', 'a', 'corporation']
['Mo', '##s', 'is', 'a', 'corporation']
['Ka', '##eno', '##n', 'is', 'a', 'corporation']
['Go', '##p', '##ro', 'is', 'a', 'corporation']
['Rev', '##ive', '##d', 'is', 'a', 'corporation']
['Canon', 'is', 'a', 'corporation']
['Pi', '##l', '##z', 'is', 'a', 'corporation']
['Come', '

['Domain', '##to', '##ols', 'is', 'a', 'corporation']
['T', '##ru', 'is', 'a', 'corporation']
['Ta', '##ser', 'is', 'a', 'corporation']
['C', '##b', '##n', 'is', 'a', 'corporation']
['Alien', '##ware', 'is', 'a', 'corporation']
['Blu', '##nds', '##tone', 'is', 'a', 'corporation']
['A', '##c', '##me', 'is', 'a', 'corporation']
['Nov', '##us', 'is', 'a', 'corporation']
['En', '##light', '##ened', 'is', 'a', 'corporation']
['So', '##yle', '##nt', 'is', 'a', 'corporation']
['Cha', '##co', 'is', 'a', 'corporation']
['White', '##wall', '##s', 'is', 'a', 'corporation']
['Vic', '##on', 'is', 'a', 'corporation']
['Partners', 'is', 'a', 'corporation']
['In', '##st', '##inet', 'is', 'a', 'corporation']
['Next', '##gen', 'is', 'a', 'corporation']
['Polar', '##is', 'is', 'a', 'corporation']
['Di', '##re', '##xi', '##on', 'is', 'a', 'corporation']
['S', '##pect', '##ro', 'is', 'a', 'corporation']
['Co', '##rt', '##o', 'is', 'a', 'corporation']
['Me', '##phi', '##sto', 'is', 'a', 'corporation']
['Le'

['Ha', '##gg', '##ar', 'is', 'a', 'corporation']
['Fritz', 'is', 'a', 'corporation']
['Braun', 'is', 'a', 'corporation']
['Op', '##pen', '##heimer', '##fu', '##nds', 'is', 'a', 'corporation']
['Um', '##bria', 'is', 'a', 'corporation']
['Atlantis', 'is', 'a', 'corporation']
['Go', '##ya', 'is', 'a', 'corporation']
['College', '##cho', '##ice', 'is', 'a', 'corporation']
['Fresh', '##pet', 'is', 'a', 'corporation']
['Morning', '##star', 'is', 'a', 'corporation']
['Ok', '##i', 'is', 'a', 'corporation']
['Aberdeen', 'is', 'a', 'corporation']
['S', '##ling', '##o', 'is', 'a', 'corporation']
['Co', '##m', '##cast', 'is', 'a', 'corporation']
['North', '##wave', 'is', 'a', 'corporation']
['Bad', '##ger', 'is', 'a', 'corporation']
['Ten', '##er', '##gy', 'is', 'a', 'corporation']
['Kid', '##de', 'is', 'a', 'corporation']
['Ku', '##ra', 'is', 'a', 'corporation']
['Di', '##ag', '##rap', '##h', 'is', 'a', 'corporation']
['Speedway', 'is', 'a', 'corporation']
['Pan', '##as', '##onic', 'is', 'a', 'co

['North', '##land', 'is', 'a', 'corporation']
['M', '##gic', 'is', 'a', 'corporation']
['Ju', '##ven', 'is', 'a', 'corporation']
['Am', '##way', 'is', 'a', 'corporation']
['Home', '##lo', '##ans', 'is', 'a', 'corporation']
['Meadows', 'is', 'a', 'corporation']
['C', '##lar', '##is', '##oy', 'is', 'a', 'corporation']
['Earth', '##link', 'is', 'a', 'corporation']
['Polo', 'is', 'a', 'corporation']
['Sony', 'is', 'a', 'corporation']
['P', '##bs', 'is', 'a', 'corporation']
['Da', '##hua', 'is', 'a', 'corporation']
['Re', '##aal', 'is', 'a', 'corporation']
['O', '##m', '##nia', 'is', 'a', 'corporation']
['Bo', '##ric', '##ua', 'is', 'a', 'corporation']
['A', '##f', '##p', 'is', 'a', 'corporation']
['Hanover', 'is', 'a', 'corporation']
['R', '##itz', 'is', 'a', 'corporation']
['Fr', '##icker', '##s', 'is', 'a', 'corporation']
['R', '##c', '##n', 'is', 'a', 'corporation']
['Sun', '##up', 'is', 'a', 'corporation']
['Shen', '##tel', 'is', 'a', 'corporation']
['T', '##p', '##g', 'is', 'a', 'corp

...

Windstar is a cruise ship travel corporation
(2, 768)
Acer is a desktop computers corporation
(2, 768)
Pims is a advertising marketing printing services corporation
(1, 768)
Venice is a ice cream frozen novelties sherbet corporation
(3, 768)
Tmz is a tv networks stations production companies corporation
(3, 768)
Kasasa is a loan credit products services corporation
(2, 768)
Vivo is a internet service providers corporation
(3, 768)
Vaseline is a skin care products corporation
(2, 768)
Sango is a dinnerware glassware pottery corporation
(2, 768)
Corona is a beer ale corporation
(3, 768)
Coronado is a cocoa milk additives corporation
(3, 768)
Purina is a cat dog food corporation
(3, 768)
Ignite is a nutritional supplements corporation
(3, 768)
Mediamath is a advertising marketing printing services corporation
(2, 768)
Hive is a security fire safety systems corporation
(2, 768)
Maurer is a vehicles sales leasing corporation
(4, 768)
Arvig is a internet service providers corporation
(3, 768

(2, 768)
Castaway is a fishing tackle equipment corporation
(3, 768)
Sperry is a sneakers corporation
(2, 768)
Vicks is a cold cough sinus asthma remedies corporation
(3, 768)
Citibank is a credit cards corporation
(3, 768)
Amalfi is a women shoes boots corporation
(2, 768)
Roys is a casual dining restaurants corporation
(1, 768)
Trinity is a security fire safety systems corporation
(1, 768)
Disney is a credit cards corporation
(2, 768)
Converse is a sportswear athleticwear corporation
(3, 768)
Moringa is a nutritional supplements corporation
(3, 768)
Sizzler is a casual dining restaurants corporation
(2, 768)
Cogo is a cocoa milk additives corporation
(3, 768)
Ripple is a security fire safety systems corporation
(2, 768)
Fila is a sportswear athleticwear corporation
(4, 768)
Aacsb is a religious charitable humanitarian corporation
(3, 768)
Australis is a women scents fragrances perfumes corporation
(2, 768)
Cues is a advertising marketing printing services corporation
(1, 768)
Doctors

(2, 768)
Sunline is a fishing tackle equipment corporation
(3, 768)
Sdsu is a ice cream frozen novelties sherbet corporation
(3, 768)
Virb is a web design domain hosting services corporation
(2, 768)
Plano is a fishing tackle equipment corporation
(2, 768)
Norms is a casual dining restaurants corporation
(3, 768)
Huebsch is a laundry equipment corporation
(3, 768)
Noritake is a dinnerware glassware pottery corporation
(3, 768)
Holmatro is a security fire safety systems corporation
(2, 768)
Behr is a paint corporation
(2, 768)
Covers is a sportswear athleticwear corporation
(2, 768)
Etro is a handbags corporation
(3, 768)
Symm is a investment products services corporation
(3, 768)
Whirl is a cooking oils corporation
(2, 768)
Pines is a nutritional supplements corporation
(3, 768)
Polident is a toothpaste whiteners corporation
(2, 768)
Mickeys is a beer ale corporation
(3, 768)
Popsicle is a ice cream frozen novelties sherbet corporation
(2, 768)
Cartier is a sunglasses corporation
(2, 7

Mottahedeh is a dinnerware glassware pottery corporation
(3, 768)
Orajel is a toothpaste whiteners corporation
(4, 768)
Electrolux is a laundry equipment corporation
(3, 768)
Breyers is a ice cream frozen novelties sherbet corporation
(2, 768)
Tods is a women shoes boots corporation
(4, 768)
Hyphen is a mattresses box springs corporation
(3, 768)
Smuckers is a peanut butter sandwich spreads corporation
(2, 768)
Mindspace is a advertising marketing printing services corporation
(4, 768)
Xylitol is a artificial sweeteners corporation
(3, 768)
Mikasa is a dinnerware glassware pottery corporation
(3, 768)
Elemis is a skin care products corporation
(3, 768)
Skittles is a candy mints corporation
(3, 768)
Portillos is a casual dining restaurants corporation
(2, 768)
Jambu is a women shoes boots corporation
(2, 768)
Artland is a dinnerware glassware pottery corporation
(3, 768)
Cubavera is a sportswear athleticwear corporation
(2, 768)
Hims is a nutritional supplements corporation
(4, 768)
Bir

Givenchy is a handbags corporation
(2, 768)
Poplin is a sportswear athleticwear corporation
(2, 768)
Dansko is a women shoes boots corporation
(2, 768)
Maxmara is a handbags corporation
(2, 768)
Elend is a loan credit products services corporation
(2, 768)
Dior is a women shoes boots corporation
(3, 768)
Athene is a investment products services corporation
(4, 768)
Stuckeys is a casual dining restaurants corporation
(2, 768)
Bushnell is a sporting goods corporation
(3, 768)
Lululemon is a sportswear athleticwear corporation
(4, 768)
Cuddledown is a bedding linens corporation
(4, 768)
Naturopathica is a skin care products corporation
(2, 768)
Wholesome is a artificial sweeteners corporation
(3, 768)
Aquafest is a cruise ship travel corporation
(2, 768)
Boppy is a bedding linens corporation
(2, 768)
Reeses is a ice cream frozen novelties sherbet corporation
(2, 768)
Sexy is a fragrances corporation
(2, 768)
Mels is a casual dining restaurants corporation
(4, 768)
Rainshadow is a fishing 

In [65]:
print(pearsonr(df_brands['alignment'], df_brands['bert_alignment']))
print(pearsonr(df_brands['alignment'], df_brands['corp_alignment']))
print(pearsonr(df_brands['alignment'], df_brands['corp_cat_alignment']))
print(pearsonr(df_brands['alignment'], df_brands['cat_bert_alignment']))
print(pearsonr(df_brands['alignment'], df_brands['cat_corp_alignment']))
print(pearsonr(df_brands['alignment'], df_brands['cat_corp_cat_alignment']))

print(pearsonr(df_brands['bert_alignment'], df_brands['corp_alignment']))
print(pearsonr(df_brands['bert_alignment'], df_brands['corp_cat_alignment']))
print(pearsonr(df_brands['corp_alignment'], df_brands['corp_cat_alignment']))
print(pearsonr(df_brands['cat_bert_alignment'], df_brands['cat_corp_alignment']))
print(pearsonr(df_brands['cat_bert_alignment'], df_brands['cat_corp_cat_alignment']))
print(pearsonr(df_brands['cat_corp_alignment'], df_brands['cat_corp_cat_alignment']))

(-0.026576595159319406, 0.30838246542259107)
(0.04410838500257011, 0.09081750151074537)
(0.09476879338932057, 0.0002728174180409922)
(-0.005395117738034939, 0.8362067584897452)
(0.036478276248827726, 0.16200801646036161)
(0.06777142765214424, 0.009320760405084463)
(0.5936470821850912, 8.726822713973405e-141)
(0.3698710365384343, 6.616286514875812e-49)
(0.566082140384084, 2.1476171380205433e-125)
(0.6570399509275328, 1.725137484958495e-182)
(0.5325029823163447, 1.6639012379364454e-108)
(0.8464138065436443, 0.0)


In [90]:
print(df_brands[df_brands['alignment'] > 0].shape[0] / df_brands.shape[0] )

print(df_brands[df_brands['bert_alignment'] > 0].shape[0] / df_brands.shape[0] )

print(df_brands[df_brands['corp_alignment'] > 0].shape[0] / df_brands.shape[0] )
print(df_brands[df_brands['corp_cat_alignment'] > 0].shape[0] / df_brands.shape[0] )
print(df_brands[df_brands['cat_bert_alignment'] > 0].shape[0] / df_brands.shape[0] )
print(df_brands[df_brands['cat_corp_alignment'] > 0].shape[0] / df_brands.shape[0] )
print(df_brands[df_brands['cat_corp_cat_alignment'] > 0].shape[0] / df_brands.shape[0] )


0.7403127124405167
0.3201903467029232
0.06934058463630184
0.02991162474507138
0.8042148198504419
0.7872195785180149
0.7688647178789939


In [73]:
web_df = pd.read_csv('../Data/brand_website_data_jan.csv', header=0)[['Name', 'Category', 'Overall Prediction', 'Gender in Photos Ratio (M:F)']]
web_df['mal'] = web_df['Gender in Photos Ratio (M:F)'].apply(lambda x: int(x.split(":")[0]))
web_df['fal'] = web_df['Gender in Photos Ratio (M:F)'].apply(lambda x: int(x.split(":")[1]))
# ratio_web_df['alignment'] = ratio_web_df['fal'] / ratio_web_df['mal']
web_df['alignment'] = (web_df['fal'] - web_df['mal']) / web_df[['fal', 'mal']].max(axis=1)


I0531 20:30:10.798326 35000 utils.py:141] NumExpr defaulting to 8 threads.


In [81]:
sort_web_df = web_df.sort_values(by='Name')
sort_bert_df = df_brands[df_brands['brand'].isin(sort_web_df['Name'].tolist())].sort_values(by='brand')

In [87]:
base_correlation = pearsonr(sort_web_df['alignment'], sort_bert_df['alignment'])
bert_correlation = pearsonr(sort_web_df['alignment'], sort_bert_df['bert_alignment'])
corp_correlation = pearsonr(sort_web_df['alignment'], sort_bert_df['corp_alignment'])
category_corp_correlation = pearsonr(sort_web_df['alignment'], sort_bert_df['corp_cat_alignment'])
residual_bert_correlation = pearsonr(sort_web_df['alignment'], sort_bert_df['cat_bert_alignment'])
residual_corp_correlation = pearsonr(sort_web_df['alignment'], sort_bert_df['cat_corp_alignment'])
residual_category_corp_correlation = pearsonr(sort_web_df['alignment'], sort_bert_df['cat_corp_cat_alignment'])

In [89]:
print(f"Correlation of Image Gender Ratio and Raw Word2Vec Embeddings: {np.round(base_correlation[0], 4)}, p-value: {np.round(base_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Bert Embeddings: {np.round(bert_correlation[0], 4)}, p-value: {np.round(bert_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Bert + Corporation Embeddings: {np.round(corp_correlation[0], 4)}, p-value: {np.round(corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Bert + Corporation + Category Embeddings: {np.round(category_corp_correlation[0], 4)}, p-value: {np.round(category_corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Bert Embeddings: {np.round(residual_bert_correlation[0], 4)}, p-value: {np.round(residual_bert_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Bert + Corporation Embeddings: {np.round(residual_corp_correlation[0], 4)}, p-value: {np.round(residual_corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Bert + Corporation + Category Embeddings: {np.round(residual_category_corp_correlation[0], 4)}, p-value: {np.round(residual_category_corp_correlation[1], 4)}")


Correlation of Image Gender Ratio and Raw Word2Vec Embeddings: 0.4357, p-value: 0.0001
Correlation of Image Gender Ratio and Bert Embeddings: 0.0633, p-value: 0.5769
Correlation of Image Gender Ratio and Bert + Corporation Embeddings: 0.2578, p-value: 0.021
Correlation of Image Gender Ratio and Bert + Corporation + Category Embeddings: 0.4047, p-value: 0.0002
Correlation of Image Gender Ratio and Residual Bert Embeddings: 0.0033, p-value: 0.9765
Correlation of Image Gender Ratio and Residual Bert + Corporation Embeddings: 0.1399, p-value: 0.2157
Correlation of Image Gender Ratio and Residual Bert + Corporation + Category Embeddings: 0.2645, p-value: 0.0178
